# 순환신경망 RNN

- 입력과 출력을 시퀀스 단위로 처리하는 모델
- 가장 기본적인 시퀀스 모델
- 활성화 함수를 통해 나온 결과값을 출력층 방향 + 은닉층 노드의 다음 계산의 입력으로 보내는 특성

- one-to-many : 하나의 입력에 대해서 여러개의 출력 이미지 캡셔닝
- many-to-one : 여러 입력에 하나의 출력 긍부정 판별 sentiment classification
- many-to-many : 여러 입력에 여러 출력 챗봇, 번역

- 하나의 은닉층에서는 가중치를 공유, 2개이상의 은닉층 은닉층 간의 가중치는 다름

- hidden_size = output_dim
- input_shape = (timesteps, input_dim)
    - timesteps : 입력 시퀀스의 길이
    - input_dim : 입력의 크기

- return_sequences
    - False : 마지막 상태만 전달 : many-to-one
    - True : 모든 시점에 대해서 은닉 상태값을 출력 : many-to-many

- RNN
    - 첫번째 입력값이 셀 A에서 처리 - h0 출력     두번째 셀로 hidden state전달
    - 두번째 입력값이 셀 A에서 처리 - h1 출력     세번째 셀로 hidden state전달
    - 사실 셀은 하나 (모든 셀은 파라미터를 공유하기 떄문에)
    - 셀이 가진 인풋 weight : Wx
    - 셀이 가진 hidden state weight : Wh   모든 셀은 같은 Wx, Wh
    

- tensorflow 가 rnn의 입력으로 받는 데이터
    - batch_size, sequence_length, input_dimension
    
- input으로 (1,1,4) 를 넣고 hidden_size=2 라면 output은 (1,1,2)

- outputs : 전체 시퀀스에 해당하는 hidden states 값들을 가지고 있음
- states : 시퀀스의 마지막 hidden state 값만 가짐

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import SimpleRNN

In [ ]:
h = [1, 0, 0, 0]
e = [0, 1, 0, 0]
l = [0, 0, 1, 0]
o = [0, 0, 0, 1]
x_data = np.array([[h, e, l, o]], dtype=np.float32)

hidden_size = 2
rnn = SimpleRNN(units=2, return_sequences=True, return_state=True)
outputs, state = rnn(x_data)

In [ ]:
x_data

In [ ]:
outputs

In [ ]:
state

In [ ]:
# 시퀀스가 5인 데이터 3개로 구성된 mini_batch

x_data =np.array([
    [h,e,l,l,o],
    [e,o,l,l,l],
    [l,l,e,e,l]
], dtype=np.float32)
hidden_size = 2
rnn = SimpleRNN(units=2, return_sequences=True, return_state=True)
outputs, states = rnn(x_data)

In [ ]:
outputs

In [ ]:
states

# Many-to-One

In [ ]:
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Embedding, SimpleRNN, Dense
from tensorflow.keras.initializers import Constant

In [ ]:
words = ['good', 'bad', 'worse', 'so good']
y_data = [1,0,0,1]

char_set = ['<pad>'] + sorted(list(set(''.join(words))))
idx2char = {idx : char for idx, char in enumerate(char_set)}
char2idx = {char : idx for idx, char in enumerate(char_set)}

print(char_set)
print(idx2char)
print(char2idx)
# 왜 pad ?
# batch 단위 연산을 하기위해서!

In [ ]:
x_data = list(map(lambda word : [char2idx.get(char) for char in word], words))
x_data_len = list(map(lambda word : len(word), x_data))

print(x_data)
print(x_data_len)

In [ ]:
max_sequence = 10
x_data = pad_sequences(sequences=x_data, maxlen=max_sequence, padding='post', truncating='post')

In [ ]:
x_data

In [ ]:
input_dim = len(char2idx)
output_dim = len(char2idx)
one_hot = np.eye(len(char2idx))
hidden_size = 10
num_classes = 2

In [ ]:
one_hot

In [ ]:
def create_M2O_model():
    model = Sequential()
    model.add(Embedding(
        input_dim=input_dim,
        output_dim=output_dim,
        trainable=False,
        mask_zero=True,
        input_length=max_sequence,
        embeddings_initializer=Constant(one_hot)
    ))
    model.add(SimpleRNN(
        units=hidden_size
    ))
    model.add(Dense(
        units=num_classes
    ))
    return model

In [ ]:
model = create_M2O_model()

In [ ]:
model.summary()